<!-- Inspired by the https://colab.research.google.com/github/jhavl/dkt/blob/main/Part%201/1%20Manipulator%20Kinematics.ipynb#scrollTo=dBlNufUaqAjc -->
## What does Forward Kinematics do?

Forward Kinematics is providing an non-linear mapping from the origin to the end-effector. We represent the end-effrector coodinate joint space in terms of coordinate space of the origin.

${}^{0}T(t) = \mathcal{K}(q(t))$, where $q(t)$ is a vector.


### The elementary transform sequence 

The elementatry transform sequence provides a universal way of describing transformations for the manipulator. ETS consists of sequence of the number of elementary transforms $E_i$ which can be either represent rotation or translation. Here is an example:

\begin{align*}
    \bf{E_i} =
    \left\{
    \begin{matrix}
        \bf{T}_{t_{x}}(\eta_i) \\
        \bf{T}_{t_{y}}(\eta_i) \\
        \bf{T}_{t_{z}}(\eta_i) \\
        \bf{T}_{R_{x}}(\eta_i) \\
        \bf{T}_{R_{y}}(\eta_i) \\
        \bf{T}_{R_{z}}(\eta_i) \\
    \end{matrix}
    \right.
\end{align*}

In other textbooks ETS is simply defined as ${}^{i}\bf{T}$

where the parameter ${\eta}$ is either a constant or a joint variable:

\begin{equation}
\eta_i = 
\left\{
    \begin{matrix}
        c_i \\ 
        q_j(t)
    \end{matrix}
\right.
\end{equation}

Joint variable is determined the type of the joint. Even though there are more than 4 types of joints, there are two most important ones from which all others are derived. So to simplify the calculations we can use them for now.
1. Revolute joint $\rightarrow \theta_j(t)$
2. Prismatic joint $\rightarrow  d_j(t)$


Each of the $\bf{T}$ elementary transformation values has its place in the resulting ETS. The elementary $T_i's$ are:

<img src="assets/positional-matrices.png" width="300" height="300">
<img src="assets/rotational-matrices.png" width="300" height="300">


</br>

In [2]:
# math 
import numpy as np

# robotics toolbox
import roboticstoolbox as rtb

# for transformations
import spatialmath as sm

In [6]:
# variable for rotation around x axis
rx_var = rtb.ET.Rx()

# constant for rotation around x-axis = 90deg
rx_cons = rtb.ET.Rx(np.pi / 2)

print(rx_var)
print(rx_cons)

Rx(q)
Rx(90°)


In [10]:
# We can get the transformation matrix from the rx_cons 
transform = rx_cons.A()

# Then we use spacial math to represent it in human readable format
transform_hm = sm.SE3(transform) 


print(f"Transform matrix is: \n {transform}")
print(f"Transform matrix human readable: \n{transform_hm}")


Transform matrix is: 
 [[ 1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
Transform matrix human readable: 
   1         0         0         0         
   0         0        -1         0         
   0         1         0         0         
   0         0         0         1         



In [11]:
# For the variable case we need to employ a variable into the A function 

q = np.pi / 4
transform = rx_var.A(q)

transform_hm = sm.SE3(transform)


print(f"Transform matrix is: \n {transform}")
print(f"Transform matrix human readable: \n{transform_hm}")

Transform matrix is: 
 [[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678  0.        ]
 [ 0.          0.70710678  0.70710678  0.        ]
 [ 0.          0.          0.          1.        ]]
Transform matrix human readable: 
   1         0         0         0         
   0         0.7071   -0.7071    0         
   0         0.7071    0.7071    0         
   0         0         0         1         



In [12]:
# Lets try to create y-axis variable transformation

ty_var = rtb.ET.ty()

# translating along y-axis by 0.25sm  
ty_cons = rtb.ET.ty(0.25)

print(ty_cons)
print(ty_var)

ty(0.25)
ty(q)


In [14]:
# We can calculate the transform coming from the ty using .A method 
transform = ty_cons.A()
transform_hm = sm.SE3(transform)

print(f"Resulting transform in human readable form:\n{transform_hm}")

Resulting transform in human readable form:
   1         0         0         0         
   0         1         0         0.25      
   0         0         1         0         
   0         0         0         1         



In [15]:
# Now let's try to do it with the variable 

transform = ty_var.A(0.15)
transform_hm = sm.SE3(transform)

print(f"Resulting transform in human readable form:\n{transform_hm}")

Resulting transform in human readable form:
   1         0         0         0         
   0         1         0         0.15      
   0         0         1         0         
   0         0         0         1         



## Encode 7-degree of freedom Franka manipulator 


<img src="./assets/cover.png" width="300px" />

In [37]:
# {}^{0}T_1
E1 = rtb.ET.tz(0.333)
E2 = rtb.ET.Rz()

# {}^{1}T_2
E3 = rtb.ET.Ry()

# {}^{2}T_3
E4 = rtb.ET.tz(0.316)
E5 = rtb.ET.Rz()

# {}^{3}T_4 (from 4 to 3)
E6 = rtb.ET.tx(0.0825)
E7 = rtb.ET.Ry(flip=True)

# {}^{4}T_5 
E8 = rtb.ET.tx(-0.0825)
E9 = rtb.ET.tz(0.384)
E10 = rtb.ET.Rz()

# {}^{5}T_6
E11 = rtb.ET.Ry(flip=True)

# {}^{6}T_7

E12 = rtb.ET.tx(0.088)
E13 = rtb.ET.Rx(np.pi)
E14 = rtb.ET.tz(0.107)
E15 = rtb.ET.Rz()



# There are many ways of how to creat ETC from now. 
#  T2 to T0
ets1 = E1 * E2 * E3 
ets2 = E4 * E5 * E6 * E7 

# E = ets1 * ets2 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15
E = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13 * E14 * E15

print(E)
print(f"Manipulator has {E.n} joints")
print(f"Manipulator has {E.m} ETs")
print(f"The first ET is manipulator ETS is {E[1]}")


print(f"The first variable joint has has a jindex: {E[1].jindex}, while the second has jindex: {E[2].jindex}")

# we can extract all the variables from the model as a list
print(f"\nAll variables in ETS: \n{E.joints()}")

tz(0.333) ⊕ Rz(q0) ⊕ Ry(q1) ⊕ tz(0.316) ⊕ Rz(q2) ⊕ tx(0.0825) ⊕ Ry(-q3) ⊕ tx(-0.0825) ⊕ tz(0.384) ⊕ Rz(q4) ⊕ Ry(-q5) ⊕ tx(0.088) ⊕ Rx(180°) ⊕ tz(0.107) ⊕ Rz(q6)
Manipulator has 7 joints
Manipulator has 15 ETs
The first ET is manipulator ETS is Rz(q0)
The first variable joint has has a jindex: 0, while the second has jindex: 1

All variables in ETS: 
[ET.Rz(jindex=0), ET.Ry(jindex=1), ET.Rz(jindex=2), ET.Ry(jindex=3, flip=True), ET.Rz(jindex=4), ET.Ry(jindex=5, flip=True), ET.Rz(jindex=6)]


## Forward kinematics

Forward kinematics is the first and most basic relationship between link geometry and robot configuration. Forward Kinematics is providing an non-linear mapping ${}^{0}T(t) = \mathcal{K}(q(t))$ between the joint space and Cartesian task space. Here ${}^{0}T_{e} \in SE(3)$ is a homogeneous trasformation matrix. 

In [38]:
def print_et(et):
    print(sm.SE3(et))

In [39]:
# Now let's populate our not defined q's in the ETS with some values 

q = np.array([0, -0.3, 0, -2.2, 0, 2, 0.79])

# allocate array 
forward_kinematics = np.eye(4)

for et in E:
    if et.isjoint:
        forward_kinematics = forward_kinematics @ et.A(q[et.jindex])
    else:
        forward_kinematics = forward_kinematics @ et.A()

print_et(forward_kinematics)

   0.7003   -0.7068    0.09983   0.4737    
  -0.7104   -0.7038    0         0         
   0.07027  -0.07092  -0.995     0.5155    
   0         0         0         1         



In [43]:
# We can use method of the ETS class instead of writing it by ourselves

# fkine calculates in human readable form, without direct involvement of spatialmath
print(f"The fkine method:\n{E.fkine(q)}")

# eval calculated raw result matrix

print(f"The eval method:\n{E.eval(q)}")

The fkine method:
   0.7003   -0.7068    0.09983   0.4737    
  -0.7104   -0.7038    0         0         
   0.07027  -0.07092  -0.995     0.5155    
   0         0         0         1         

The eval method:
[[ 7.00329021e-01 -7.06804465e-01  9.98334166e-02  4.73724040e-01]
 [-7.10353272e-01 -7.03845316e-01 -1.22464680e-16 -1.31037208e-17]
 [ 7.02672827e-02 -7.09169942e-02 -9.95004165e-01  5.15513206e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
